In [2]:
%cd turnus

[WinError 2] The system cannot find the file specified: 'turnus'
m:\Documents\FRI\phd\rl-optimization\turnus


In [3]:
import torch
import pandas as pd
import numpy as np

def load_node_csv(path: str, index_col: str, encoders=None, **kwargs):
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)
        df = df[encoders.keys()].copy()

    return x, mapping, df

def build_edge_connections(time_matrix_path: str, start_times: np.array, finish_times: np.array, delimiter=';'):
    matrix = np.loadtxt(time_matrix_path, delimiter=delimiter)
    # Add the transport times (matrix) to the finishing time of every task
    transport_times = matrix + np.expand_dims(finish_times, axis=-1)

    possible_connection = np.where(transport_times <= start_times)

    return torch.tensor(np.array(possible_connection))


In [4]:
class IdentityEncoder:
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

In [5]:
# Normalize and convert dataframe column (Series) into a tensor
class NumberNormEncoder:
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(((df - df.mean()) / df.std()).values).view(-1, 1).to(self.dtype)

In [6]:
from torch_geometric.data import Data
import torch_geometric.transforms as T

nodes, mapping, df = load_node_csv('data/1/tasks.csv', 'Index', 
                               {name: NumberNormEncoder() for name in ['ZastavkaStart', 'ZastavkaFinish', 'CasStart', 'CasFinish', 'Vzdialenost']}, 
                               sep=';')

# extra_features = torch.zeros((nodes.shape[0], 1))

# nodes = torch.cat((nodes, extra_features), 1)

edge_connections = build_edge_connections('data/1/Tij.csv', df['CasStart'].values, df['CasFinish'].values)

data = Data(x=nodes, edge_index=edge_connections)

c = T.Constant(0)


data = c(data)
data.x[0, 5] = 1

data

C:\Users\David\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)


Data(x=[51, 6], edge_index=[2, 1195])

In [43]:
data.node_stores

[{'x': tensor([[-4.1161, -3.8783, -1.4623, -1.5567, -3.6148,  0.0000],
         [ 0.7066, -0.1244, -1.2705, -1.2716,  0.4634,  0.0000],
         [ 0.7066, -0.1244, -1.2233, -1.2240,  0.4634,  0.0000],
         [ 0.7066, -0.1244, -1.2076, -1.2082,  0.4634,  0.0000],
         [-0.0972,  0.7818, -1.1888, -1.1987, -0.0700,  0.0000],
         [-0.2312,  0.7818, -1.1699, -1.1670,  0.4072,  0.0000],
         [-0.2312,  0.7818, -1.1070, -1.1036,  0.4072,  0.0000],
         [ 0.7066, -0.1244, -1.0976, -1.0973,  0.4634,  0.0000],
         [ 0.7066, -0.1244, -1.0504, -1.0498,  0.4634,  0.0000],
         [-0.2312,  0.7818, -1.0127, -1.0086,  0.4072,  0.0000],
         [ 0.7066, -0.1244, -0.9970, -0.9959,  0.4634,  0.0000],
         [-0.2312,  0.7818, -0.9812, -0.9769,  0.4072,  0.0000],
         [-0.2312,  0.7818, -0.9184, -0.9135,  0.4072,  0.0000],
         [ 0.7066, -0.1244, -0.9089, -0.9072,  0.4634,  0.0000],
         [-0.2312,  0.7818, -0.8806, -0.8755,  0.4072,  0.0000],
         [ 0.7066, 

In [7]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [8]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [16]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [19]:
data.num_edge_features

0

In [10]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8180
